In [1]:
import ray

In [2]:
if ray.is_initialized():
    ray.shutdown()
ray.init()

2025-09-14 15:05:03,559	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.44.1
Dashboard:,http://127.0.0.1:8265


(reduce pid=1657662) Error calculating size for column 'title': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(reduce pid=1657662) Error calculating size for column 'description': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(reduce pid=1657662) Error calculating size for column 'tag': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(reduce pid=1657662) Error calculating size for column 'title': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(reduce pid=1657662) Error calculating size for column 'description': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(reduce pid=1657662) Error calculating size for column 'tag': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(reduce pid=1657679) Error calculating size for column 'title': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(reduce pid=1657679) Error calculating size for column 'description': cannot call `vectorize` on

In [3]:
ray.cluster_resources()

{'node:__internal_head__': 1.0,
 'node:172.27.14.77': 1.0,
 'CPU': 96.0,
 'object_store_memory': 200000000000.0,
 'accelerator_type:RTX': 1.0,
 'memory': 876914446336.0,
 'GPU': 1.0}

## Finetunning our own llm

In [4]:
import os
import random
import torch
import numpy as np
import ray
from ray.data.preprocessor import Preprocessor

In [5]:
def set_seed(seed=42):
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #eval("setattr(torch.backends.cudnn, 'deterministic', True)")
    #eval("setattr(torch.backends.cudnn, 'benchmark', False)")
    os.environ['PYTHONHASHSEED'] = str(seed)

In [6]:
set_seed()

In [7]:
DATASET_LOC = "https://raw.githubusercontent.com/GokuMohandas/Made-With-ML/main/datasets/dataset.csv"

In [8]:
def load_data(num_samples=None, loc=DATASET_LOC):
    ds = ray.data.read_csv(loc)
    ds = ds.random_shuffle(seed=1234)
    ds = ray.data.from_items(ds.take(num_samples)) if num_samples else ds
    return ds

<div style="border-left: 4px solid #00c896; background-color:rgb(23, 22, 22); padding: 1em; border-radius: 8px; margin: 1em 0;">
  <p style="margin: 0; font-weight: bold; color: #059669;">💡 Tip</p>
  <p style="margin: 0;">
    When working with very large datasets, it's a good idea to limit the number of samples in our dataset so that we can execute our code quickly and iterate on bugs, etc.  
    This is why we have a <code>num_samples</code> input argument in our <code>load_data</code> function (<code>None</code> = no limit, all samples).
  </p>
</div>


## Helpful functions

In [9]:
import numpy as np
from transformers import BertTokenizer
import re
import nltk
from nltk.corpus import stopwords

In [10]:
nltk.download("stopwords")
STOPWORDS = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /work/ngkuissi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
tokenizer = BertTokenizer.from_pretrained("allenai/scibert_scivocab_uncased", return_dict=False)

/mnt/hpc/work/ngkuissi/Made-With-ML/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
def tokenize(batch, tokenizer=tokenizer):
    encoded_inputs = tokenizer(batch['text'].tolist(), return_tensors="np", padding="longest")
    return dict(ids=encoded_inputs['input_ids'], mask=encoded_inputs['attention_mask'],
                target=np.array(batch['tag']))

In [13]:
def clean_text(text, stopwords=STOPWORDS):
    """Clean raw text string."""
    text = text.lower()
    
    # remove stopwords
    pattern = re.compile(r'\b(' + r"|".join(stopwords) + r")\b\s*")
    text = pattern.sub("", text)
    
    text = re.sub(r"([!\"'#$%&()*\+,-./:;<=>?@\\\[\]^_`{|}~])", r" \1 ", text)  # add spacing
    text = re.sub("[^A-Za-z0-9]+", " ", text)  # remove non alphanumeric chars
    text = re.sub(" +", " ", text)  # remove multiple spaces
    text = text.strip()  # strip white space at the ends
    text = re.sub(r"http\S+", "", text)  #  remove links
    
    return text
    

In [14]:
def preprocess(df, class_to_index):
    """Preprocess the data."""
    df["text"] = df.title + " " + df.description  # feature engineering
    df["text"] = df.text.apply(clean_text)  # clean text
    df = df.drop(columns=["id", "created_on", "title", "description"], errors="ignore")  # clean dataframe
    df = df[["text", "tag"]]  # rearrange columns
    df["tag"] = df["tag"].map(class_to_index)  # label encoding
    outputs = tokenize(df)
    return outputs

In [15]:
class CustomPreprocessor(Preprocessor):
    
    def _fit(self, ds):
        tags = ds.unique(column="tag")
        self.class_to_index = {tag: i for i, tag in enumerate(tags)}
        self.index_to_class = {v:k for k, v in self.class_to_index.items()}
    
    def _transform_pandas(self, batch):
        return preprocess(batch, class_to_index=self.class_to_index)

In [16]:
def decode(indices, index_to_class):
    return [index_to_class[idx] for idx in indices]

## Model

In [17]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

In [18]:
llm = BertModel.from_pretrained("allenai/scibert_scivocab_uncased", return_dict=False)
embedding_dim = llm.config.hidden_size

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
text = "Transfer learning with transformers for text classification."
batch = tokenizer([text], return_tensors="pt", padding="longest")
seq, pool = llm(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])

In [20]:
seq.shape

torch.Size([1, 10, 768])

In [21]:
pool.shape

torch.Size([1, 768])

In [22]:
class FinetunnedLLM(nn.Module):
    
    def __init__(self, llm, dropout_p, embedding_dim, num_classes):
        super(FinetunnedLLM, self).__init__()
        self.llm = llm
        self.dropout = nn.Dropout(dropout_p)
        self.classifier = nn.Linear(embedding_dim, num_classes)
    
    def forward(self, batch):
        ids, mask = batch['ids'], batch['mask']
        _, pool = self.llm(input_ids=ids, attention_mask = mask)
        
        return self.classifier(self.dropout(pool))

    @torch.inference_mode
    def predict(self, batch):
        self.eval()
        z = self(batch)
        return torch.argmax(z, dim=-1).cpu().numpy()

    @torch.inference_mode
    def predict_prob(self, batch):
        self.eval()
        z = self(batch)
        y_probs = F.softmax(z, dim=-1).cpu().numpy()
        return y_probs

In [23]:
model = FinetunnedLLM(llm, 0.5, embedding_dim, 4)

## Data setup

In [24]:
ray.data.DatasetContext.get_current().execution_options.preserve_order = True
ds = ray.data.read_csv(DATASET_LOC)
ds = ds.random_shuffle(seed=1234)

In [25]:
import sys
sys.path.append("..")
from madewithml.data import stratify_split

/mnt/hpc/work/ngkuissi/Made-With-ML/venv/lib/python3.10/site-packages/mlflow/utils/requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [26]:
test_size = 0.2
train_ds, val_ds = stratify_split(ds, stratify="tag", test_size=test_size)

In [27]:
tags = train_ds.unique(column="tag")
class_to_index = {tag: i for i, tag in enumerate(tags)}

2025-09-14 15:05:20,524	INFO dataset.py:2796 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2025-09-14 15:05:20,529	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:05:20,530	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> AllToAllOperator[Sort] -> AllToAllOperator[MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- RandomShuffle 2: 0.00 row [00:00, ? row/s]

Shuffle Map 3:   0%|                                                                                          …

...

Sort Sample 13:   0%|                                                                                         …

Shuffle Map 14:   0%|                                                                                         …

Shuffle Reduce 15:   0%|                                                                                      …

- limit=1 16: 0.00 row [00:00, ? row/s]

2025-09-14 15:05:20,806	WARNING progress_bar.py:120 -- Truncating long operator name to 100 characters. To disable this behavior, set `ray.data.DataContext.get_current().DEFAULT_ENABLE_PROGRESS_BAR_NAME_TRUNCATION = False`.


In [28]:
sample_ds = train_ds.map_batches(
    preprocess,
    fn_kwargs={"class_to_index": class_to_index},
    batch_format="pandas"
)

## Batching

In [29]:
from ray.train.torch import get_device

Created a temporary directory at /tmp/tmpevnbdfei
Writing /tmp/tmpevnbdfei/_remote_module_non_scriptable.py


In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [31]:
def pad_array(arr, dtype=np.int32):
    max_len = max([len(row) for row in arr])
    padded_arr = np.zeros((arr.shape[0], max_len), dtype=dtype)
    for i, row in enumerate(arr):
        padded_arr[i, :len(row)]= row
    return padded_arr

In [32]:
def collate_fn(batch, device=device):
    batch["ids"] = pad_array(batch["ids"])
    batch["mask"] = pad_array(batch["mask"])
    dtypes = {"ids": torch.int32, "mask": torch.int32, "target": torch.int64}
    tensor_batch = {}
    for key, arr in batch.items():
        tensor_batch[key] = torch.as_tensor(list(arr), dtype=dtypes[key], device=device)
    return tensor_batch
    

In [36]:
sample_batch = sample_ds.take_batch(128)

2025-09-14 15:05:42,828	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:05:42,828	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> AllToAllOperator[Sort] -> AllToAllOperator[MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle] -> TaskPoolMapOperator[MapBatches(preprocess)] -> LimitOperator[limit=128]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- RandomShuffle 2: 0.00 row [00:00, ? row/s]

Shuffle Map 3:   0%|                                                                                          …

...

- MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle 9: 0.00 row [00:00, ? row/s]

Shuffle Map 10:   0%|                                                                                         …

Shuffle Reduce 11:   0%|                                                                                      …

- MapBatches(preprocess) 12: 0.00 row [00:00, ? row/s]

- limit=128 13: 0.00 row [00:00, ? row/s]

In [37]:
collate_fn(sample_batch)

{'ids': tensor([[  102,  1438,  4706,  ...,     0,     0,     0],
         [  102,  2389, 29477,  ...,     0,     0,     0],
         [  102, 24638, 15781,  ...,     0,     0,     0],
         ...,
         [  102,  1613, 15037,  ...,     0,     0,     0],
         [  102,  1572,  8039,  ...,     0,     0,     0],
         [  102,   856,  3997,  ...,     0,     0,     0]], device='cuda:0',
        dtype=torch.int32),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.int32),
 'target': tensor([1, 2, 3, 2, 0, 0, 0, 2, 2, 1, 0, 1, 2, 2, 1, 2, 3, 2, 0, 0, 3, 2, 2, 0,
         0, 2, 2, 2, 2, 3, 2, 2, 0, 0, 1, 0, 0, 3, 3, 2, 2, 0, 0, 2, 2, 0, 0, 1,
         0, 0, 3, 2, 2, 0, 0, 2, 2, 1, 3, 0, 0, 3, 0, 0, 2, 2, 0, 0, 1, 0, 0, 3,
         2, 2, 0, 0, 2, 2, 3, 2, 2, 0, 0, 2, 2, 1, 3, 2,

## Utilities

In [38]:
from ray.air import session
from ray.air.config import CheckpointConfig, DatasetConfig, RunConfig, ScalingConfig
from ray.train.torch import prepare_model
from ray.train.torch import TorchCheckpoint, TorchTrainer
import torch.nn.functional as F

In [39]:
def train_step(ds, model, batch_size, num_classes, loss_fn, optimizer):
    model.train()
    loss = 0.0
    ds_generator = ds.iter_torch_batches(batch_size=batch_size, collate_fn=collate_fn)
    for i, batch in enumerate(ds_generator):
        optimizer.zero_grad()
        z = model(batch)
        targets = batch['target']
        e_loss = loss_fn(z.view(-1, z.shape[-1]), targets)
        e_loss.backward()
        optimizer.step()
        loss += (e_loss.detach().item() - loss) / (i + 1)
    return loss

In [40]:
def eval_step(ds, batch_size, model, num_classes, loss_fn):
    model.eval()
    loss = 0.0
    y_trues, y_preds = [], []
    ds_generator = ds.iter_torch_batches(batch_size=batch_size, collate_fn=collate_fn)
    
    with torch.inference_mode():
        for i, batch in enumerate(ds_generator):
            z = model(batch)
            targets = batch['target']
            e_loss = loss_fn(z.view(-1, z.shape[-1]), targets).item()
            loss += (e_loss - loss) / (i + 1)
            y_trues.extend(batch['target'].cpu().numpy())
            y_preds.extend(torch.argmax(z, dim=-1).cpu().numpy())
            
    return loss, np.vstack(y_trues), np.vstack(y_preds)
            

In [41]:
def train_loop_per_worker(config):
    dropout = config['dropout_p']
    lr = config['lr']
    lr_factor = config['lr_factor']
    lr_patience = config['lr_patience']
    num_epochs = config['num_epochs']
    batch_size = config['batch_size']
    num_classes = config['num_classes']
    device = config['device']
    
    #set_seed()
    train_ds = session.get_dataset_shard("train")
    val_ds = session.get_dataset_shard("val")
    
    llm = BertModel.from_pretrained("allenai/scibert_scivocab_uncased", return_dict=False).cuda()
    model = FinetunnedLLM(llm, dropout, llm.config.hidden_size, num_classes)
    model = prepare_model(model)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=lr_factor, patience=lr_patience)
    
    batch_size_per_worker = batch_size // session.get_world_size()
    for epoch in range(num_epochs):
        train_loss = train_step(train_ds, model, batch_size_per_worker, num_classes, loss_fn, optimizer)
        val_loss, _, _ = eval_step(val_ds, batch_size_per_worker, model, num_classes, loss_fn)
        scheduler.step(val_loss)
        
        metrics = dict(epoch=epoch, lr=optimizer.param_groups[0]["lr"], train_loss=train_loss, val_loss=val_loss)
        checkpoint = TorchCheckpoint.from_state_dict(model.state_dict())
        session.report(metrics, checkpoint=checkpoint)
    

In [42]:
train_loop_config = {
    "dropout_p": 0.5,
    "lr": 1e-4,
    "lr_factor": 0.8,
    "lr_patience": 3,
    "num_epochs": 10,
    "batch_size": 128,
    "num_classes": 4,
    "device": device
}

In [43]:
num_workers = 1
resources_per_worker = {"CPU": 10, "GPU": 1}

In [44]:
from ray.train import ScalingConfig

scaling_config = ScalingConfig(
    num_workers=num_workers,
    use_gpu=bool(resources_per_worker.get("GPU", 0)),
    resources_per_worker=resources_per_worker,
)

In [45]:
checkpoint_config = CheckpointConfig(num_to_keep=1, checkpoint_score_attribute="val_loss", checkpoint_score_order="min")

In [46]:
run_config = RunConfig(name="llm", storage_path="~/ray_results", checkpoint_config=checkpoint_config)

## Training

In [47]:
ds = load_data()
train_ds, val_ds = stratify_split(ds, stratify="tag", test_size=test_size)

In [48]:
preprocessor = CustomPreprocessor()
train_ds = preprocessor.fit_transform(train_ds)
val_ds = preprocessor.transform(val_ds)

2025-09-14 15:06:03,115	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:06:03,116	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> AllToAllOperator[Sort] -> AllToAllOperator[MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- RandomShuffle 2: 0.00 row [00:00, ? row/s]

Shuffle Map 3:   0%|                                                                                          …

...

- Aggregate 12: 0.00 row [00:00, ? row/s]

Sort Sample 13:   0%|                                                                                         …

Shuffle Map 14:   0%|                                                                                         …

Shuffle Reduce 15:   0%|                                                                                      …

- limit=1 16: 0.00 row [00:00, ? row/s]

In [49]:
train_ds = train_ds.materialize()
val_ds = val_ds.materialize()

2025-09-14 15:06:08,418	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:06:08,418	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> AllToAllOperator[Sort] -> AllToAllOperator[MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle] -> TaskPoolMapOperator[CustomPreprocessor]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- RandomShuffle 2: 0.00 row [00:00, ? row/s]

Shuffle Map 3:   0%|                                                                                          …

...

Shuffle Reduce 8:   0%|                                                                                       …

- MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle 9: 0.00 row [00:00, ? row/s]

Shuffle Map 10:   0%|                                                                                         …

Shuffle Reduce 11:   0%|                                                                                      …

- CustomPreprocessor 12: 0.00 row [00:00, ? row/s]

In [50]:
from ray.train import DataConfig

In [51]:
dataset_config = DataConfig(["train", "val"])

In [52]:
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config=train_loop_config,
    scaling_config=scaling_config,
    run_config= run_config,
    datasets= {"train": train_ds, "val": val_ds},
    dataset_config=dataset_config,
    #preprocessor=preprocessor   
)

In [53]:
results = trainer.fit()

2025-09-14 15:06:25,113	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2025-09-14 15:06:25 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-06-25/llm/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-09-14 15:06:30 (running for 00:00:05.13)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-06-25/llm/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-09-14 15:06:35 (running for 00:00:10.16)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-06-25/llm/driver_artifacts
Number of trials

(pid=1667582) Running 0: 0.00 row [00:00, ? row/s]

(pid=1667582) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

== Status ==
Current time: 2025-09-14 15:06:40 (running for 00:00:15.18)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-06-25/llm/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




...

(pid=1667582) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

(pid=1667581) Running 0: 0.00 row [00:00, ? row/s]

(pid=1667581) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

2025-09-14 15:07:13,982	WARNING experiment_state.py:206 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds and may become a bottleneck. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this warning by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0). Set it to 0 to completely suppress this warning.
2025-09-14 15:07:15,160	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/work/ngkuissi/ray_results/llm' in 0.0084s.
2025-09-14 15:07:15,162	INFO tune.py:1041 -- Total run time: 50.05 seconds (49.88 seconds for the tuning loop).


== Status ==
Current time: 2025-09-14 15:07:15 (running for 00:00:49.89)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-06-25/llm/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)




In [54]:
results.metrics_dataframe.columns

Index(['epoch', 'lr', 'train_loss', 'val_loss', 'timestamp',
       'checkpoint_dir_name', 'should_checkpoint', 'done',
       'training_iteration', 'trial_id', 'date', 'time_this_iter_s',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore',
       'iterations_since_restore', 'config/train_loop_config/dropout_p',
       'config/train_loop_config/lr', 'config/train_loop_config/lr_factor',
       'config/train_loop_config/lr_patience',
       'config/train_loop_config/num_epochs',
       'config/train_loop_config/batch_size',
       'config/train_loop_config/num_classes',
       'config/train_loop_config/device'],
      dtype='object')

In [55]:
results.metrics_dataframe[['epoch', 'train_loss', 'val_loss']]

,epoch,train_loss,val_loss
0,0,1.338384,1.178855
1,1,1.122036,0.756896
2,2,0.664604,0.560707
3,3,0.382146,0.616958
4,4,0.252215,0.629421
5,5,0.162674,0.421725
6,6,0.083551,0.479942
7,7,0.029594,0.556606
8,8,0.015381,0.590856
9,9,0.010082,0.618089


In [56]:
results.best_checkpoints

[(Checkpoint(filesystem=local, path=/work/ngkuissi/ray_results/llm/TorchTrainer_e8dd0_00000_0_2025-09-14_15-06-25/checkpoint_000005),
  {'epoch': 5,
   'lr': 0.0001,
   'train_loss': 0.1626741498708725,
   'val_loss': 0.4217247813940048,
   'timestamp': 1757876818,
   'checkpoint_dir_name': 'checkpoint_000005',
   'should_checkpoint': True,
   'done': False,
   'training_iteration': 6,
   'trial_id': 'e8dd0_00000',
   'date': '2025-09-14_15-07-00',
   'time_this_iter_s': 3.517345428466797,
   'time_total_s': 28.58060359954834,
   'pid': 1667252,
   'hostname': 'gpu-pt1-04',
   'node_ip': '172.27.14.77',
   'config': {'train_loop_config': {'dropout_p': 0.5,
     'lr': 0.0001,
     'lr_factor': 0.8,
     'lr_patience': 3,
     'num_epochs': 10,
     'batch_size': 128,
     'num_classes': 4,
     'device': 'cuda'}},
   'time_since_restore': 28.58060359954834,
   'iterations_since_restore': 6})]

In [57]:
from ray.train.torch import TorchPredictor
from sklearn.metrics import precision_recall_fscore_support

In [58]:
llm = BertModel.from_pretrained("allenai/scibert_scivocab_uncased", return_dict=False).cuda()
model = FinetunnedLLM(llm, 0.5, llm.config.hidden_size, 4)

/mnt/hpc/work/ngkuissi/Made-With-ML/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expect

In [59]:
results.best_checkpoints

[(Checkpoint(filesystem=local, path=/work/ngkuissi/ray_results/llm/TorchTrainer_e8dd0_00000_0_2025-09-14_15-06-25/checkpoint_000005),
  {'epoch': 5,
   'lr': 0.0001,
   'train_loss': 0.1626741498708725,
   'val_loss': 0.4217247813940048,
   'timestamp': 1757876818,
   'checkpoint_dir_name': 'checkpoint_000005',
   'should_checkpoint': True,
   'done': False,
   'training_iteration': 6,
   'trial_id': 'e8dd0_00000',
   'date': '2025-09-14_15-07-00',
   'time_this_iter_s': 3.517345428466797,
   'time_total_s': 28.58060359954834,
   'pid': 1667252,
   'hostname': 'gpu-pt1-04',
   'node_ip': '172.27.14.77',
   'config': {'train_loop_config': {'dropout_p': 0.5,
     'lr': 0.0001,
     'lr_factor': 0.8,
     'lr_patience': 3,
     'num_epochs': 10,
     'batch_size': 128,
     'num_classes': 4,
     'device': 'cuda'}},
   'time_since_restore': 28.58060359954834,
   'iterations_since_restore': 6})]

In [60]:
best_checkpoint = results.best_checkpoints[0][0].path + "/model.pt"
state_dict = torch.load(best_checkpoint)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [61]:
model

FinetunnedLLM(
  (llm): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [62]:
HOLDOUT_LOC = "https://raw.githubusercontent.com/GokuMohandas/Made-With-ML/main/datasets/holdout.csv"
test_ds = ray.data.read_csv(HOLDOUT_LOC)
preprocessed_ds = preprocessor.transform(test_ds)

In [63]:
preprocessed_ds = preprocessor.transform(test_ds)
values = preprocessed_ds.select_columns(cols=["target"]).take_all()
y_true = np.stack([item["target"] for item in values])

2025-09-14 15:07:42,716	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:07:42,717	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> TaskPoolMapOperator[CustomPreprocessor->Project]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- CustomPreprocessor->Project 2: 0.00 row [00:00, ? row/s]

In [64]:
y_true

array([2, 2, 2, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 2, 3, 3, 0, 0, 2, 0, 2, 3,
       2, 0, 3, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 1, 1, 0, 3,
       3, 2, 1, 0, 3, 0, 0, 2, 2, 3, 0, 2, 0, 2, 2, 2, 2, 3, 3, 3, 3, 0,
       0, 3, 0, 1, 3, 0, 2, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 3, 1, 3, 3, 3,
       3, 2, 3, 3, 0, 3, 0, 0, 2, 0, 3, 0, 3, 0, 3, 2, 3, 3, 3, 3, 3, 0,
       3, 3, 0, 0, 0, 0, 2, 3, 0, 3, 0, 3, 0, 2, 2, 2, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 3, 2, 3, 0, 0, 1, 1, 0, 3,
       3, 3, 3, 3, 3, 0, 0, 0, 3, 0, 1, 1, 0, 3, 3, 1, 0, 2, 0, 0, 0, 3,
       3, 0, 3, 0, 3, 2, 3, 0, 0, 3, 1, 0, 1, 0, 0])

In [65]:
model = model.to("cuda")

In [66]:
ds_generator = preprocessed_ds.iter_torch_batches(batch_size=128, collate_fn=collate_fn)
y_pred = None
model.eval()
with torch.inference_mode():
    for i, batch in enumerate(ds_generator):
        z = model(batch)
        if y_pred is not None:
            y_pred = torch.concat([y_pred, torch.argmax(z, dim=-1)], dim=0)
        else:
            y_pred = torch.argmax(z, dim=-1)

2025-09-14 15:07:48,473	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:07:48,473	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> TaskPoolMapOperator[CustomPreprocessor]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- CustomPreprocessor 2: 0.00 row [00:00, ? row/s]

In [67]:
y_pred = y_pred.cpu().numpy()

In [68]:
metrics = precision_recall_fscore_support(y_true, y_pred, average="weighted")

In [69]:
{"precision": metrics[0], "recall": metrics[1], "f1": metrics[2]}

{'precision': 0.934616372140792,
 'recall': 0.9319371727748691,
 'f1': 0.9322420348871521}

In [70]:
def evaluate(ds, model, preprocessor=preprocessor, device="cuda"):
    # y_true
    preprocessed_ds = preprocessor.transform(ds)
    values = preprocessed_ds.select_columns(cols=["target"]).take_all()
    y_true = np.stack([item["target"] for item in values])
    
    # y_pred
    model = model.to(device)
    ds_generator = preprocessed_ds.iter_torch_batches(batch_size=128, collate_fn=lambda x: collate_fn(x, device=device))
    y_pred = None
    model.eval()
    with torch.inference_mode():
        for i, batch in enumerate(ds_generator):
            z = model(batch)
            if y_pred is not None:
                y_pred = torch.concat([y_pred, torch.argmax(z, dim=-1)], dim=0)
            else:
                y_pred = torch.argmax(z, dim=-1)
    
    y_pred = y_pred.cpu().numpy()
    metrics = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    return {"precision": metrics[0], "recall": metrics[1], "f1": metrics[2]}

In [71]:
evaluate(test_ds, model)

2025-09-14 15:08:02,483	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:08:02,484	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> TaskPoolMapOperator[CustomPreprocessor->Project]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- CustomPreprocessor->Project 2: 0.00 row [00:00, ? row/s]

2025-09-14 15:08:04,698	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:08:04,698	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> TaskPoolMapOperator[CustomPreprocessor]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- CustomPreprocessor 2: 0.00 row [00:00, ? row/s]

{'precision': 0.934616372140792,
 'recall': 0.9319371727748691,
 'f1': 0.9322420348871521}

## Inference

In [72]:
import pandas as pd

In [73]:
preprocessor.index_to_class

{0: 'natural-language-processing',
 1: 'mlops',
 2: 'other',
 3: 'computer-vision'}

In [74]:
def format_prob(prob, index_to_class):
    d = {}
    for i, item in enumerate(prob):
        d[index_to_class[i]] = item
    return d

In [75]:
def predict_prob(df, model, preprocessor=preprocessor, device="cuda"):
    
    processed = preprocessor._transform_pandas(df)
    processed = collate_fn(processed, device)
    model = model.to(device)
    output = model(processed)
    output = output.cpu().detach()
    y_prob = output.softmax(dim=1).numpy()
    
    res = []
    for i, prob in enumerate(y_prob):
        tag = decode([prob.argmax()], preprocessor.index_to_class)[0]
        res.append({"prediction": tag, "probabilities": format_prob(prob, preprocessor.index_to_class)})
    return res
    

In [76]:
title = "Transfer learning with transformers"
description = "Using transformers for transfer learning on text classification tasks."
sample_df = pd.DataFrame([{"title": title, "description": description, "tag": "natural-language-processing"}])

In [77]:
predict_prob(sample_df, model)

[{'prediction': 'natural-language-processing',
  'probabilities': {'natural-language-processing': 0.9966356,
   'mlops': 0.0011847689,
   'other': 0.0008976348,
   'computer-vision': 0.0012820732}}]

# MLFLOW

In [78]:
import mlflow
from pathlib import Path
from ray.air.integrations.mlflow import MLflowLoggerCallback
import time

In [79]:
MODEL_REGISTERY = Path("tmp/mlflow")
MODEL_REGISTERY.mkdir(exist_ok=True, parents=True)

In [80]:
MLFLOW_TRACKING_URI = "file://" + str(MODEL_REGISTERY.absolute())
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [81]:
print(mlflow.get_tracking_uri())

file:///mnt/hpc/work/ngkuissi/Made-With-ML/tmp/mlflow


In [82]:
experiment_name = f"llm-{int(time.time())}"
mlflow_callback = MLflowLoggerCallback(
    mlflow.get_tracking_uri(),
    experiment_name=experiment_name,
    save_artifact=True
)

In [83]:
run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    callbacks=[mlflow_callback]
)

In [84]:
ds = load_data()
train_ds, val_ds = stratify_split(ds, stratify="tag", test_size=test_size)

In [85]:
preprocessor = CustomPreprocessor()
train_ds = preprocessor.fit_transform(train_ds)
val_ds = preprocessor.transform(val_ds)

2025-09-14 15:44:07,027	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:44:07,028	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> AllToAllOperator[Sort] -> AllToAllOperator[MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- RandomShuffle 2: 0.00 row [00:00, ? row/s]

Shuffle Map 3:   0%|                                                                                          …

...

- Aggregate 12: 0.00 row [00:00, ? row/s]

Sort Sample 13:   0%|                                                                                         …

Shuffle Map 14:   0%|                                                                                         …

Shuffle Reduce 15:   0%|                                                                                      …

- limit=1 16: 0.00 row [00:00, ? row/s]

In [86]:
train_ds = train_ds.materialize()
val_ds = val_ds.materialize()

2025-09-14 15:44:21,439	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/logs/ray-data
2025-09-14 15:44:21,439	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> AllToAllOperator[RandomShuffle] -> AllToAllOperator[Sort] -> AllToAllOperator[MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle] -> TaskPoolMapOperator[CustomPreprocessor]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(192) 1: 0.00 row [00:00, ? row/s]

- RandomShuffle 2: 0.00 row [00:00, ? row/s]

Shuffle Map 3:   0%|                                                                                          …

...

Shuffle Reduce 8:   0%|                                                                                       …

- MapBatches(_add_split)->MapBatches(_filter_split)->RandomShuffle 9: 0.00 row [00:00, ? row/s]

Shuffle Map 10:   0%|                                                                                         …

Shuffle Reduce 11:   0%|                                                                                      …

- CustomPreprocessor 12: 0.00 row [00:00, ? row/s]

In [87]:
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config=train_loop_config,
    scaling_config=scaling_config,
    run_config= run_config,
    datasets= {"train": train_ds, "val": val_ds},
    dataset_config=dataset_config,
    #preprocessor=preprocessor   
)

/mnt/hpc/work/ngkuissi/Made-With-ML/venv/lib/python3.10/site-packages/ray/train/base_trainer.py:582: RayDeprecationWarning: `ray.train.RunConfig(callbacks: List[ray.tune.Callback])` is deprecated. Ray Train no longer accepts Ray Tune callbacks, since the Ray Train execution backend is being separated from Ray Tune. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0
  _log_deprecation_warning(CALLBACKS_DEPRECATION_MESSAGE)


In [88]:
results = trainer.fit()

2025-09-14 15:45:04,680	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2025-09-14 15:45:04 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-45-04/TorchTrainer_2025-09-14_15-45-04/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-09-14 15:45:09 (running for 00:00:05.13)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-45-04/TorchTrainer_2025-09-14_15-45-04/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-09-14 15:45:14 (running for 00:00:10.18)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts

(pid=1706393) Running 0: 0.00 row [00:00, ? row/s]

(pid=1706393) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

== Status ==
Current time: 2025-09-14 15:45:19 (running for 00:00:15.20)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-45-04/TorchTrainer_2025-09-14_15-45-04/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




...

(pid=1706392) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

2025-09-14 15:45:58,814	WARNING experiment_state.py:206 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds and may become a bottleneck. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this warning by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0). Set it to 0 to completely suppress this warning.


copying /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-45-04/TorchTrainer_2025-09-14_15-45-04/driver_artifacts/TorchTrainer_4f6eb_00000_0_2025-09-14_15-45-04/params.json -> /mnt/hpc/work/ngkuissi/Made-With-ML/tmp/mlflow/240405567451470118/bf9b0f0f771142718785a853aa1a9ce1/artifacts
copying /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-45-04/TorchTrainer_2025-09-14_15-45-04/driver_artifacts/TorchTrainer_4f6eb_00000_0_2025-09-14_15-45-04/events.out.tfevents.1757879111.gpu-pt1-04 -> /mnt/hpc/work/ngkuissi/Made-With-ML/tmp/mlflow/240405567451470118/bf9b0f0f771142718785a853aa1a9ce1/artifacts
copying /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-45-04/TorchTrainer_2025-09-14_15-45-04/driver_artifacts/TorchTrainer_4f6eb_00000_0_2025-09-14_15-45-04/params.pkl -> /mnt/hpc/work/ngkuissi/Made-With-ML/tmp/mlflow/240405567451470118/bf9b0f0f771142718785a853aa1a9ce1/artifacts
copying /tmp/ray/session_2025-09-14_15

2025-09-14 15:46:00,050	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/work/ngkuissi/ray_results/TorchTrainer_2025-09-14_15-45-04' in 0.0067s.
2025-09-14 15:46:00,051	INFO tune.py:1041 -- Total run time: 55.37 seconds (55.34 seconds for the tuning loop).


== Status ==
Current time: 2025-09-14 15:46:00 (running for 00:00:55.34)
Using FIFO scheduling algorithm.
Logical resource usage: 11.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:RTX)
Result logdir: /tmp/ray/session_2025-09-14_15-05-00_735286_1657229/artifacts/2025-09-14_15-45-04/TorchTrainer_2025-09-14_15-45-04/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)




In [89]:
results.metrics_dataframe

,epoch,lr,train_loss,val_loss,timestamp,checkpoint_dir_name,should_checkpoint,done,training_iteration,trial_id,...,time_since_restore,iterations_since_restore,config/train_loop_config/dropout_p,config/train_loop_config/lr,config/train_loop_config/lr_factor,config/train_loop_config/lr_patience,config/train_loop_config/num_epochs,config/train_loop_config/batch_size,config/train_loop_config/num_classes,config/train_loop_config/device
0,0,0.00010,1.279368,1.173194,1757879121,checkpoint_000000,True,False,1,4f6eb_00000,...,11.715830,1,0.5,0.0001,0.8,3,10,128,4,cuda
1,1,0.00010,1.120310,0.860466,1757879125,checkpoint_000001,True,False,2,4f6eb_00000,...,14.729786,2,0.5,0.0001,0.8,3,10,128,4,cuda
2,2,0.00010,0.703693,0.606365,1757879128,checkpoint_000002,True,False,3,4f6eb_00000,...,18.123309,3,0.5,0.0001,0.8,3,10,128,4,cuda
3,3,0.00010,0.347788,0.533715,1757879132,checkpoint_000003,True,False,4,4f6eb_00000,...,21.727601,4,0.5,0.0001,0.8,3,10,128,4,cuda
4,4,0.00010,0.145502,0.667248,1757879136,checkpoint_000004,True,False,5,4f6eb_00000,...,25.387019,5,0.5,0.0001,0.8,3,10,128,4,cuda
5,5,0.00010,0.102597,0.448974,1757879140,checkpoint_000005,True,False,6,4f6eb_00000,...,29.349467,6,0.5,0.0001,0.8,3,10,128,4,cuda
6,6,0.00010,0.047516,0.656775,1757879144,checkpoint_000006,True,False,7,4f6eb_00000,...,33.376119,7,0.5,0.0001,0.8,3,10,128,4,cuda
7,7,0.00010,0.032291,0.535943,1757879148,checkpoint_000007,True,False,8,4f6eb_00000,...,37.371826,8,0.5,0.0001,0.8,3,10,128,4,cuda
8,8,0.00010,0.014956,0.641851,1757879152,checkpoint_000008,True,False,9,4f6eb_00000,...,40.921743,9,0.5,0.0001,0.8,3,10,128,4,cuda
9,9,0.00008,0.005292,0.835750,1757879156,checkpoint_000009,True,False,10,4f6eb_00000,...,44.951023,10,0.5,0.0001,0.8,3,10,128,4,cuda


In [92]:
sorted_runs = mlflow.search_runs(experiment_names=experiment_name,
                                        order_by=['metrics.val_loss ASC'])

2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name l
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name l
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name m
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name -
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name 1
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name 7
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name 5
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name 7
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name 8
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name 7
2025/09/14 15:48:23 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name 8
2025/09/14 15:48:23 W

In [93]:
sorted_runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time
